In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
# Introduction
# If you want to find the good hangout place and have more number of coffee shops.
#The problem we aim to solve is to analyze the coffee shops locations in the major US cities and find the best hangout place.

In [52]:
# FourSquare API  will be used to collect data about locations of Coffee shops in Central Toronto and North York

Comparing Two Neigborhood for living and commercial facilities. Need to Enter Foursquare ID and Client KEY.

In [2]:
#Obtain Postal Code, Borough, and Neighborhood information from Wikipedia
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

#Obtain the first table
df_toronto = table[0]
df_toronto.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [3]:
df_toronto.rename(columns = {"Postal code": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace = True)

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)
#df.head()

#Combine the neighborhoods that exists in one postal code
df_toronto = df_toronto.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
#df.head()

#Change unassigned Neighborhood to its Borough's name
df_toronto.loc[85,'Neighborhood'] = 'Queen\'s Park'

print (df_toronto.shape)

df_toronto.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
#Create a dataframe of the latitude and longitudes of the Toronto Neighborhoods
latlong = pd.read_csv("http://cocl.us/Geospatial_data")
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
latlong.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
#Join the Lat and Long dataframe to Neighborhoods dataframe
df_toronto.set_index("PostalCode")
latlong.set_index("PostalCode")
neighbor=pd.merge(df_toronto, latlong)
neighbor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [7]:
neighbor['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [8]:
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

C:\Users\swati\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


In [9]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbor['Latitude'], neighbor['Longitude'], neighbor['Borough'], neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

In [10]:
map_toronto

In [11]:
central_toronto_data = neighbor[neighbor['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,Moore Park / Summerhill East,43.689574,-79.383160


In [12]:
address = 'Central Toronto, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto, CA are {}, {}.'.format(latitude, longitude))

C:\Users\swati\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Central Toronto, CA are 43.6534817, -79.3839347.


In [13]:
map_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(central_toronto_data['Latitude'], central_toronto_data['Longitude'], central_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central_toronto)  

In [14]:
map_central_toronto

In [15]:
CLIENT_ID = 'OH4Q3JMYB1AQI04VI30NXNKZR2332ZJMXAH2TOBKPQNX2LDT' # your Foursquare ID
CLIENT_SECRET = 'PKK42YJDUYXQOX4YGMDJUAS1PATKLYUE4FBUAYC2ZQTEJQSI' # your Foursquare Secret
VERSION = '20200101' #Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OH4Q3JMYB1AQI04VI30NXNKZR2332ZJMXAH2TOBKPQNX2LDT
CLIENT_SECRET:PKK42YJDUYXQOX4YGMDJUAS1PATKLYUE4FBUAYC2ZQTEJQSI


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [17]:
central_toronto_venues = getNearbyVenues(names=central_toronto_data['Neighborhood'],
                                   latitudes=central_toronto_data['Latitude'],
                                   longitudes=central_toronto_data['Longitude']
                                    )

Found 116 venues in 9 neighborhoods.


In [18]:
print(central_toronto_venues.shape)
central_toronto_venues.head()

(116, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,J.N. Munkongolo & Associates - Family Lawyer,43.727437,-79.390264,Lawyer
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop


In [19]:
#Venues per Neighborhood
central_toronto_venues.groupby('Neighborhood').count()
central_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,J.N. Munkongolo & Associates - Family Lawyer,43.727437,-79.390264,Lawyer
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop


In [20]:
#one-hot vector for venue category
central_toronto_one_hot = pd.get_dummies(central_toronto_venues['Venue Category'])
central_toronto_one_hot['Neighborhood'] = central_toronto_venues['Neighborhood']
central_toronto_one_hot.head()

,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,...,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Lawrence Park
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Lawrence Park
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Lawrence Park
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Lawrence Park
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Davisville North


In [21]:
central_toronto_mean_venue = central_toronto_one_hot.groupby('Neighborhood').mean().reset_index()
central_toronto_mean_venue.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,...,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.0,0.0,0.0,0.0,0.027778,0.000000,0.027778,0.0,0.00,...,0.000000,0.0,0.055556,0.00,0.027778,0.027778,0.00,0.0,0.0,0.0
1,Davisville North,0.0,0.0,0.0,0.0,0.000000,0.142857,0.000000,0.0,0.00,...,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0
2,Forest Hill North & West,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.25,...,0.000000,0.0,0.250000,0.00,0.000000,0.000000,0.25,0.0,0.0,0.0
3,Lawrence Park,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.25,...,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.00,0.0,0.0,0.0
4,Moore Park / Summerhill East,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,...,0.333333,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0


In [22]:
#Top 10 venues of Central Toronto
num_top_venues = 10
for hood in central_toronto_mean_venue['Neighborhood']:
    print("----"+hood+"----")
    temp = central_toronto_mean_venue[central_toronto_mean_venue['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0         Pizza Place  0.11
1        Dessert Shop  0.08
2      Sandwich Place  0.08
3                Café  0.06
4         Coffee Shop  0.06
5    Sushi Restaurant  0.06
6                 Gym  0.06
7  Italian Restaurant  0.06
8          Restaurant  0.03
9  Seafood Restaurant  0.03


----Davisville North----
                 venue  freq
0       Sandwich Place  0.14
1     Department Store  0.14
2                Hotel  0.14
3       Breakfast Spot  0.14
4    Food & Drink Shop  0.14
5                 Park  0.14
6                  Gym  0.14
7           Playground  0.00
8  Rental Car Location  0.00
9                  Pub  0.00


----Forest Hill North & West----
                       venue  freq
0              Jewelry Store  0.25
1                      Trail  0.25
2           Sushi Restaurant  0.25
3                   Bus Line  0.25
4         Light Rail Station  0.00
5               Liquor Store  0.00
6         Mexican Restaurant  0.00
7  Middle Ea

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = central_toronto_mean_venue['Neighborhood']

for ind in np.arange(central_toronto_mean_venue.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_toronto_mean_venue.iloc[ind, :], num_top_venues)

In [25]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Café,Coffee Shop,Italian Restaurant,Gym,Indoor Play Area,Pub
1,Davisville North,Department Store,Food & Drink Shop,Hotel,Gym,Sandwich Place,Breakfast Spot,Park,Donut Shop,Farmers Market,Fast Food Restaurant
2,Forest Hill North & West,Bus Line,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop
3,Lawrence Park,Bus Line,Lawyer,Swim School,Park,Yoga Studio,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Garden
4,Moore Park / Summerhill East,Summer Camp,Playground,Park,Yoga Studio,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Department Store
5,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Seafood Restaurant,Fast Food Restaurant,Diner,Mexican Restaurant,Dessert Shop,Park,Rental Car Location
6,Roselawn,Garden,Pool,Yoga Studio,Hotel,Health & Beauty Service,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint
7,Summerhill West / Rathnelly / South Hill / For...,Pub,Coffee Shop,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant
8,The Annex / North Midtown / Yorkville,Café,Sandwich Place,Coffee Shop,Indian Restaurant,Pub,BBQ Joint,Burger Joint,Cheese Shop,Cosmetics Shop,Donut Shop


In [26]:
neighborhoods_venues_sorted.iloc[8,]

Neighborhood              The Annex / North Midtown / Yorkville
1st Most Common Venue                                      Café
2nd Most Common Venue                            Sandwich Place
3rd Most Common Venue                               Coffee Shop
4th Most Common Venue                         Indian Restaurant
5th Most Common Venue                                       Pub
6th Most Common Venue                                 BBQ Joint
7th Most Common Venue                              Burger Joint
8th Most Common Venue                               Cheese Shop
9th Most Common Venue                            Cosmetics Shop
10th Most Common Venue                               Donut Shop
Name: 8, dtype: object

In [27]:
# set number of clusters
kclusters = 4

central_toronto_grouped_clustering = central_toronto_mean_venue.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(central_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe 
kmeans.labels_

array([0, 0, 3, 0, 2, 0, 1, 0, 0])

In [28]:
central_toronto = central_toronto_data
central_toronto['Clusters'] = kmeans.labels_ 
central_toronto = central_toronto.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')
central_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Lawyer,Swim School,Park,Yoga Studio,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Garden
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Department Store,Food & Drink Shop,Hotel,Gym,Sandwich Place,Breakfast Spot,Park,Donut Shop,Farmers Market,Fast Food Restaurant
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Clothing Store,Coffee Shop,Yoga Studio,Seafood Restaurant,Fast Food Restaurant,Diner,Mexican Restaurant,Dessert Shop,Park,Rental Car Location
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Café,Coffee Shop,Italian Restaurant,Gym,Indoor Play Area,Pub
4,M4T,Central Toronto,Moore Park / Summerhill East,43.689574,-79.383160,2,Summer Camp,Playground,Park,Yoga Studio,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Department Store
5,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,43.686412,-79.400049,0,Pub,Coffee Shop,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Pool,Yoga Studio,Hotel,Health & Beauty Service,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint
7,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,0,Bus Line,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop
8,M5R,Central Toronto,The Annex / North Midtown / Yorkville,43.672710,-79.405678,0,Café,Sandwich Place,Coffee Shop,Indian Restaurant,Pub,BBQ Joint,Burger Joint,Cheese Shop,Cosmetics Shop,Donut Shop


In [29]:
# create map
map_clusters_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(central_toronto['Latitude'], central_toronto['Longitude'],central_toronto['Neighborhood'], central_toronto['Clusters']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_central_toronto)

In [30]:
map_clusters_central_toronto

In [31]:
#Another Part of City for comparison
north_york_data = neighbor[neighbor['Borough'] == 'North York'].reset_index(drop=True)
north_york_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,Fairview / Henry Farm / Oriole,43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,York Mills / Silver Hills,43.757490,-79.374714
4,M2M,North York,Willowdale / Newtonbrook,43.789053,-79.408493


In [32]:
address = 'North York, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York, CA are {}, {}.'.format(latitude, longitude))

C:\Users\swati\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of North York, CA are 43.7543263, -79.44911696639593.


In [33]:
# create map of Scarborough using latitude and longitude values
map_north_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(north_york_data['Latitude'], north_york_data['Longitude'], north_york_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york)

In [34]:
map_north_york

In [35]:
north_york_venues = getNearbyVenues(names= north_york_data['Neighborhood'],
                                   latitudes=north_york_data['Latitude'],
                                   longitudes=north_york_data['Longitude']
                                    )

Found 249 venues in 24 neighborhoods.


In [36]:
# one hot encoding
north_york_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_york_onehot['Neighborhood'] = north_york_venues['Neighborhood'] 

In [37]:
north_york_onehot.head()

,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hillcrest Village
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hillcrest Village
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hillcrest Village
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hillcrest Village
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Fairview / Henry Farm / Oriole


In [38]:
north_york_data_grouped = north_york_onehot.groupby('Neighborhood').mean().reset_index()
north_york_data_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.090909,0.0,...,0.045455,0.045455,0.0,0.000000,0.0,0.0,0.0,0.045455,0.0,0.0
1,Bayview Village,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.250000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,Bedford Park / Lawrence Manor East,0.0,0.000000,0.041667,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.041667,0.0,0.041667,0.0,0.0,0.0,0.000000,0.0,0.0
3,Don Mills,0.0,0.000000,0.000000,0.0,0.076923,0.038462,0.0,0.000000,0.0,...,0.038462,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,Downsview,0.0,0.058824,0.000000,0.0,0.000000,0.058824,0.0,0.058824,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [39]:
num_top_venues = 10

for hood in north_york_data_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = north_york_data_grouped[north_york_data_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor / Wilson Heights / Downsview North----
                 venue  freq
0          Coffee Shop  0.09
1                 Bank  0.09
2       Sandwich Place  0.05
3  Fried Chicken Joint  0.05
4        Deli / Bodega  0.05
5    Convenience Store  0.05
6             Pharmacy  0.05
7          Pizza Place  0.05
8   Chinese Restaurant  0.05
9           Restaurant  0.05


----Bayview Village----
                       venue  freq
0         Chinese Restaurant  0.25
1                       Bank  0.25
2                       Café  0.25
3        Japanese Restaurant  0.25
4          Accessories Store  0.00
5                 Kids Store  0.00
6                       Park  0.00
7              Movie Theater  0.00
8         Miscellaneous Shop  0.00
9  Middle Eastern Restaurant  0.00


----Bedford Park / Lawrence Manor East----
                  venue  freq
0        Sandwich Place  0.08
1    Italian Restaurant  0.08
2           Coffee Shop  0.08
3            Restaurant  0.08
4           Pizza

In [40]:
north_york_data_grouped['Neighborhood'].value_counts()

Bedford Park / Lawrence Manor East                   1
Humberlea / Emery                                    1
Fairview / Henry Farm / Oriole                       1
Humber Summit                                        1
Bayview Village                                      1
Don Mills                                            1
Victoria Village                                     1
Northwood Park / York University                     1
Hillcrest Village                                    1
Downsview                                            1
Parkwoods                                            1
Bathurst Manor / Wilson Heights / Downsview North    1
York Mills / Silver Hills                            1
York Mills West                                      1
Willowdale / Newtonbrook                             1
Willowdale                                           1
Glencairn                                            1
North Park / Maple Leaf Park / Upwood Park           1
Lawrence M

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
north_york_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
north_york_neighborhoods_venues_sorted['Neighborhood'] = north_york_data_grouped['Neighborhood']

for ind in np.arange(north_york_data_grouped.shape[0]):
    north_york_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_york_data_grouped.iloc[ind, :], num_top_venues)

In [42]:
north_york_neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Bank,Gas Station,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Deli / Bodega,Ice Cream Shop,Convenience Store
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,Bedford Park / Lawrence Manor East,Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Fast Food Restaurant,Comfort Food Restaurant,Café,Butcher,Indian Restaurant,Pharmacy
3,Don Mills,Beer Store,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Chinese Restaurant,Clothing Store,Caribbean Restaurant,Café
4,Downsview,Grocery Store,Park,Construction & Landscaping,Bank,Hotel,Home Service,Gym / Fitness Center,Shopping Mall,Baseball Field,Athletics & Sports
5,Fairview / Henry Farm / Oriole,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Restaurant,Sporting Goods Shop,Bus Station,Japanese Restaurant,Bank,Convenience Store
6,Glencairn,Asian Restaurant,Pizza Place,Pub,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
7,Hillcrest Village,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Women's Store,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
8,Humber Summit,Pizza Place,Empanada Restaurant,Women's Store,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
9,Humberlea / Emery,Food Service,Baseball Field,Women's Store,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


In [43]:
# set number of clusters
kclusters = 4

north_york_clustering = north_york_data_grouped.drop('Neighborhood',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(north_york_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 2, 0])

In [44]:
#north_york = north_york_data.loc[north_york_data['Neighborhood'].isin(north_york_neighborhoods_venues_sorted['Neighborhood'])]
north_york = north_york_data.drop_duplicates(subset='Neighborhood',keep='last')
north_york

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,Fairview / Henry Farm / Oriole,43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,York Mills / Silver Hills,43.757490,-79.374714
4,M2M,North York,Willowdale / Newtonbrook,43.789053,-79.408493
6,M2P,North York,York Mills West,43.752758,-79.400049
7,M2R,North York,Willowdale,43.782736,-79.442259
8,M3A,North York,Parkwoods,43.753259,-79.329656
10,M3C,North York,Don Mills,43.725900,-79.340923
11,M3H,North York,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259


In [45]:
north_york = north_york.join(north_york_neighborhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')
north_york

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Women's Store,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
1,M2J,North York,Fairview / Henry Farm / Oriole,43.778517,-79.346556,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Restaurant,Sporting Goods Shop,Bus Station,Japanese Restaurant,Bank,Convenience Store
2,M2K,North York,Bayview Village,43.786947,-79.385975,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
3,M2L,North York,York Mills / Silver Hills,43.757490,-79.374714,Cafeteria,Women's Store,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
4,M2M,North York,Willowdale / Newtonbrook,43.789053,-79.408493,Piano Bar,Women's Store,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
6,M2P,North York,York Mills West,43.752758,-79.400049,Park,Convenience Store,Bank,Bar,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
7,M2R,North York,Willowdale,43.782736,-79.442259,Pizza Place,Ramen Restaurant,Coffee Shop,Café,Sandwich Place,Sushi Restaurant,Electronics Store,Movie Theater,Pharmacy,Pet Store
8,M3A,North York,Parkwoods,43.753259,-79.329656,Park,Food & Drink Shop,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
10,M3C,North York,Don Mills,43.725900,-79.340923,Beer Store,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Chinese Restaurant,Clothing Store,Caribbean Restaurant,Café
11,M3H,North York,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259,Coffee Shop,Bank,Gas Station,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Deli / Bodega,Ice Cream Shop,Convenience Store


In [46]:
north_york['Clusters'] = kmeans.labels_
north_york

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Clusters
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Women's Store,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,0
1,M2J,North York,Fairview / Henry Farm / Oriole,43.778517,-79.346556,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Restaurant,Sporting Goods Shop,Bus Station,Japanese Restaurant,Bank,Convenience Store,0
2,M2K,North York,Bayview Village,43.786947,-79.385975,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,0
3,M2L,North York,York Mills / Silver Hills,43.757490,-79.374714,Cafeteria,Women's Store,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,0
4,M2M,North York,Willowdale / Newtonbrook,43.789053,-79.408493,Piano Bar,Women's Store,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,0
6,M2P,North York,York Mills West,43.752758,-79.400049,Park,Convenience Store,Bank,Bar,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,0
7,M2R,North York,Willowdale,43.782736,-79.442259,Pizza Place,Ramen Restaurant,Coffee Shop,Café,Sandwich Place,Sushi Restaurant,Electronics Store,Movie Theater,Pharmacy,Pet Store,0
8,M3A,North York,Parkwoods,43.753259,-79.329656,Park,Food & Drink Shop,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,0
10,M3C,North York,Don Mills,43.725900,-79.340923,Beer Store,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Chinese Restaurant,Clothing Store,Caribbean Restaurant,Café,0
11,M3H,North York,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259,Coffee Shop,Bank,Gas Station,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Deli / Bodega,Ice Cream Shop,Convenience Store,1


In [47]:
# create map
map_clusters_north_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_york['Latitude'], north_york['Longitude'], north_york['Neighborhood'], north_york['Clusters']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_north_york)

In [48]:
map_clusters_north_york

Thus Comparing between Central Toronto and York by using Number of Neighborhoods, Venues and their clusters, it can be concluded that 
Central Torornto has better living and commercial facilities than York. 

In [49]:
#Places with Coffee Shops in North York
north_york_coffee_shops = north_york_data_grouped.loc[north_york_data_grouped['Coffee Shop'] > 0.001]

In [50]:
north_york_coffee_shops[['Neighborhood','Coffee Shop']]

,Neighborhood,Coffee Shop
0,Bathurst Manor / Wilson Heights / Downsview North,0.090909
2,Bedford Park / Lawrence Manor East,0.083333
3,Don Mills,0.076923
5,Fairview / Henry Farm / Oriole,0.071429
10,Lawrence Manor / Lawrence Heights,0.076923
12,Northwood Park / York University,0.166667
14,Victoria Village,0.250000
15,Willowdale,0.078947


In [51]:
#Places with Coffee Shops in Central Toronto
central_toronto_coffee_shops = central_toronto_mean_venue.loc[central_toronto_mean_venue['Coffee Shop'] > 0.001]
central_toronto_coffee_shops[['Neighborhood','Coffee Shop']]

,Neighborhood,Coffee Shop
0,Davisville,0.055556
5,North Toronto West,0.095238
7,Summerhill West / Rathnelly / South Hill / For...,0.125000
8,The Annex / North Midtown / Yorkville,0.086957


Thus it can be determined that North York has many Coffee Shops than Central Toronto. Thus can be good hangout place.